In [1]:

# Pandas is a package containing additional functions to use data frames in Python
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
import numpy as np
import seaborn as sns
warnings.simplefilter('ignore')
# These two lines allow the notebook to access the Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# This is the path to the project folder within the Google Drive.
file_path = "/content/drive/My Drive/"
import gc

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install geopandas
import geopandas as gpd

     |████████████████████████████████| 972kB 2.8MB/s 
     |████████████████████████████████| 10.9MB 16.8MB/s 
     |████████████████████████████████| 14.7MB 308kB/s 


In [3]:
import itertools
import os

# Overlap Maps

This code combines various things we've done previously to make maps showing how different pairs of species are predicted to overlap.

First we make lists of all the factors.

In [41]:
species_list = [line.strip() for line in open(file_path + "species_names.tsv")]
models = ['BCC-CSM2-MR',
         'CanESM5']
scenarios = ['ssp126', 'ssp585']

time_periods = ['2021-2040', '2041-2060', '2061-2080', '2081-2100']

sdm = 'rf'

# store all the combination names in a list
combos = ['present']
for model in models:
  for ssp in scenarios:
    for timepoint in time_periods:
      combos.append(model + "_" + ssp + "_" + timepoint)


We read the table generated in notebook 17 with information about species overlaps.

In [6]:
results = pd.read_csv(file_path + "overlaps.tsv", sep="\t")
results.index = np.arange(len(results))

We also need to regenerate the columns with the proportion of overlap, as I saved before I added these.

In [17]:
propcols = []
# for every combination of factors
for combo in combos:
  # divide the number of shared squares by the total number of habitable squares
  prop = results['total_shared_habitable_squares_' + combo] / results['species1_total_habitable_squares_' + combo]
  results['proportion_species1_habitat_shared_' + combo] = prop
  propcols.append('proportion_species1_habitat_shared_' + combo)

Identify cells where the percentage overlap is higher for any projection than for the present.

In [ ]:
isless = pd.DataFrame()
for i in np.arange(1, len(propcols)):
  isless[i] = results[propcols[i]] > results['proportion_species1_habitat_shared_present']

Find the rows where this is the case for at least four model / ssp / timepoint combinations.

In [42]:
# add up the number of cells with higher proportions than the present
# find the indices for the ones which are > 4 
all_less = np.where(isless.sum(1) > 4)

# get these rows from the table
increasing = results.loc[all_less]
increasing.index = range(len(increasing))

Extract all the combinations of Apis mellifera plus another species where this is the case.

In [45]:
spp = list(zip(increasing['species1'][increasing['species1'] == 'Apis_mellifera'], increasing['species2'][increasing['species1'] == 'Apis_mellifera']))

These variables are just for arranging the maps on the page and setting the titles and labels.

In [46]:
# which combinations are for each model / spp combination
blocks = [((1, 5)), ((5, 9)), ((9, 13)), ((13, 17))]

# titles for the pages of plots
titles = ['BCC-CSM2-MR_ssp126', 'BCC-CSM2-MR_ssp585', 'CanESM5_ssp126', 'CanESM5_ssp585']

# titles for the subplots
decades = ['present', '2021-2040', '2041-2060', '2061-2080', '2081-2100']

Now we can plot the maps for these combinations.

In [ ]:
# This just choses four colours from the Set3 colourmap for the map
col1 = 8
col2 = 2
col3 = 4
col4 = 3
# for every species combination with A. mellifera
for species1, species2 in spp:
  l = 0
  # take all the model / ssp combinations
  for block in blocks:
    # varibles to keep track of position on the page
    i = 1
    k = 0
    # number of rows and columns for the maps
    x = 5
    y = 3
    # make an empty figure
    f = plt.figure(figsize=(25, 20))
    for j in np.append(0, np.arange(block[0], block[1])):
        # take this model / ssp combination
        combo = combos[j]

        # I messed up the "_"s in the file names a bit - this fixes it
        if combo == "present":
          st = "_"
          title = 'present'
        else:
          st = "__"
          title = decades[k]
        
        # read in the species range predictions for A. mellifera
        path1 = file_path + "SDM_results/" + species1 + "/" + combo + st + sdm + ".npy"
        data1 = np.load(path1)

        # change the numbers in the matrix to make the colours nice
        data1[data1 == 0] = col1
        data1[data1 == 1] = col2
        
        # read in the species range predictions for the other species
        path2 = file_path + "SDM_results/" + species2 + "/" + combo + st + sdm + ".npy"
        data2 = np.load(path2)

        # change the colours for matrix 2
        data2[data2 == 0] = col1
        data2[data2 == 1] = col3
    
        # make a third matrix with different numbers depending if species1, species2 or both are present

        # make the empty matrix
        data3 = np.zeros([930, 2160])

        # extract where the sea is from matrix2 and put it into matrix 3
        data3[np.isnan(data2)] = float('nan')

        # colour 1 - areas with neither species
        data3[data3 == 0] = col1
        # colour 2 - areas with only species 1
        data3[data1 == col2] = col2
        # colour 3 - areas with only species 2
        data3[data2 == col3] = col3
        # colour 4 - areas with both species
        data3[(data1 == col2) & (data2 == col3)] = col4
        
        # add a subplot
        a = f.add_subplot(x, y, i)
        # visualise species 1
        a.imshow(data1, vmin=0, vmax=12, cmap='Set3', interpolation='nearest')
        # hide the axis
        a.set_axis_off()
        # add the title
        a.set_title(title + " " + species1.replace("_", " "))
        # move to the next subplot
        i += 1
        a = f.add_subplot(x, y, i)
        # repeat for matrix two and three
        a.imshow(data3, vmin=0, vmax=12, cmap='Set3', interpolation='nearest')
        a.set_axis_off()
        a.set_title(title + " Combined")
        i += 1
        a = f.add_subplot(x, y, i)
        a.imshow(data2, vmin=0, vmax=12, cmap='Set3', interpolation='nearest')
        a.set_axis_off()
        a.set_title(title + " " + species2.replace("_", " "))
        i += 1
        k += 1
    # set the main title
    f.suptitle(titles[l].replace("_", " ") + "\n" + species1.replace("_", " ") + " vs " + species2.replace("_", " "), y=0.95, fontsize=14)

    # save
    f.savefig(file_path + "SDM_overlap_maps/" + species1 + "_" + species2 + "_" + titles[l] + ".png", dpi=300, bbox_inches='tight')
    # clear the memory - stops it from crashing as much
    f.clf()
    plt.close()
    gc.collect()

    l += 1